In [295]:
import numpy as np 
import pandas as pd
import sys
from astropy.io import fits
import plotly.graph_objects as go
sys.path.append('../../dash_gaia_orbits/')

#'Alessi_36', 'Collinder_135', 'NGC_2451B', 'OC_0450' Package imports
from dash_gaia_orbits.dash_test import create_dash_app
from dash_gaia_orbits import StarClusterData, StarClusterCollection, StarClusters3DPlotter

column_renaming_dict = {'n_stars_hunt' : 'n_stars', 'U_new' : 'U', 'V_new' : 'V', 'W_new' : 'W', 'x_hunt_50' : 'x', 'y_hunt_50' : 'y', 'z_hunt_50' : 'z', 'U_err_new' : 'U_err', 'V_err_new' : 'V_err', 'W_err_new' : 'W_err'}

In [296]:
static_traces = []
static_traces_times = []


# read in Gordian scatter map
#dfv = pd.read_csv('/Users/cam/Downloads/edenhofer_dust_scatter_new.csv')
dfv = pd.read_csv('/Users/cam/Desktop/astro_research/radcliffe/mustache_work/figures/cubes/edenhofer_scatterized_4pc.csv')
dfv = dfv.loc[dfv['e'].between(0.003, 1)]
#dfv = dfv.loc[dfv['extinction'].between(0.0007, 1)]
ds_index = 2
scatter_edenhofer = go.Scatter3d(
    x=dfv['x'].values[::ds_index],
    y=dfv['y'].values[::ds_index],
    z=dfv['z'].values[::ds_index],
    mode='markers',
    marker=dict(size=2,
                color='gray',
                symbol='circle',
                opacity=.08),
    line = dict(color = 'gray', width = 0.),
    name='Edenhofer Dust',
    visible = True,
    hovertext='Edenhofer Dust',
    hoverinfo='skip'
    )
static_traces.append(scatter_edenhofer)
static_traces_times.append([0.]) # only show at t=0

# gsh238 shell
dfv = pd.read_csv('/Users/cam/Downloads/gsh_238_region_edenhofer.csv')
dfv = dfv.loc[dfv['extinction'].between(0.0007, 1)]
scatter_gsh238 = go.Scatter3d(
    x=dfv['X'].values,
    y=dfv['Y'].values,
    z=dfv['Z'].values,
    mode='markers',
    marker=dict(size=6,
                color='turquoise',
                symbol='circle',
                opacity=.02),
    line = dict(color = 'gray', width = 0.),
    name='GSH238',
    visible = 'legendonly',
    hovertext='GSH238')
static_traces.append(scatter_gsh238)
static_traces_times.append([0.]) # only show at t=0

# read in leike map

hdu = fits.open('/Users/cam/Downloads/Local_Bubble_Pelgrims2020.fits')
x = hdu[1].data['x'][::4]
y = hdu[1].data['y'][::4]
z = hdu[1].data['z'][::4]

scatter_lb = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(size=1,
                color='#6495ED',
                symbol='circle',
                opacity=.05),
    name='Local Bubble',
    visible = 'legendonly',
    hovertext='Local Bubble')
static_traces.append(scatter_lb)
static_traces_times.append([0.]) # only show at t=0

# static_traces.append(scatter_ivs)
# scatter_ivs_times = np.arange(0, -5., -0.5)
# static_traces_times.append(scatter_ivs_times)

In [297]:
df_hunt_full = pd.read_csv('/Users/cam/Downloads/hunt_catalog_oct_13.csv')
#df_hunt_full = pd.read_csv('/Users/cam/Downloads/hunt_sample_chronos_ages.csv')
df_hunt_full = df_hunt_full.rename(columns={'n_stars_hunt' : 'n_stars', 'U_new' : 'U', 'V_new' : 'V', 'W_new' : 'W', 'x_hunt_50' : 'x', 'y_hunt_50' : 'y', 'z_hunt_50' : 'z', 'U_err_new' : 'U_err', 'V_err_new' : 'V_err', 'W_err_new' : 'W_err'})

df_hunt_good = df_hunt_full.loc[
    (df_hunt_full['n_rv_gaia_apogee_galah'] >= 3) & 
    (df_hunt_full['U_err'] < 5) & 
    (df_hunt_full['V_err'] < 5) & 
    (df_hunt_full['W_err'] < 5)
    ]
df_hunt_young = df_hunt_good.loc[df_hunt_good['age_myr'] < 15]
df_hunt_older = df_hunt_good.loc[df_hunt_good['age_myr'].between(0, 150)]
df_hunt_older.to_csv('/Users/cam/Downloads/hunt_catalog_0_70_myr.csv', index=False)

In [298]:
df_hunt_gagne = pd.read_csv('/Users/cam/Downloads/cluster_sample_data.csv')
df_hunt_gagne = df_hunt_gagne[['name', 'family', 'age_myr', 'U', 'V', 'W', 'x_helio', 'y_helio', 'z_helio', 'n_stars']]
df_hunt_gagne = df_hunt_gagne.rename(columns={'x_helio' : 'x', 'y_helio' : 'y', 'z_helio' : 'z'})

m6 = df_hunt_gagne[df_hunt_gagne['family'] == 'm6']
cr135 = df_hunt_gagne[df_hunt_gagne['family'] == 'cr135']
ap = df_hunt_gagne[df_hunt_gagne['family'] == 'alphaPer']
ap_young = ap.loc[ap['age_myr'] < 15]
gamma_vel = df_hunt_gagne[df_hunt_gagne['family'] == 'gamma_velorum']

df_hunt_partIII = pd.read_csv('/Users/cam/Downloads/hunt_part3_clusters.csv')
df_hunt_partIII_to_merge = df_hunt_partIII[['name', 'kind']]
m6 = m6.merge(df_hunt_partIII_to_merge, on='name', how='left')

sun = pd.DataFrame({'name' : ['Sun'], 'family' : ['Sun'], 'age_myr' : [8000], 'U' : [0.], 'V' : [0.], 'W' : [0.], 'x' : [0.], 'y' : [0.], 'z' : [27.], 'n_stars' : [1]})


In [299]:
# RW/Split/CS clusters
rw_region = pd.read_csv('/Users/cam/Downloads/rw_clusters.csv')
rw = rw_region.loc[rw_region.age_myr < 15]
remove = ['Theia_6', 'Teutsch_39', 'OCSN_32', 'Aveni-Hunter_1', 'IC_5146', 'Collinder_419', 'Dolidze_8', 'Theia_1920', 'Collinder_95', 'Mon_OB1-D', 'NGC_2264', 'UBC_159']
rw = rw[~rw['name'].isin(remove)]
rw = rw.rename(columns = column_renaming_dict)

In [300]:
# omuamua
x,y,z = 0., 0., 27.
U, V, W = -11.622, -22.461, -7.824
# U, V, W = 33.1, -6.8, 8.3
name = 'Oumuamua'
age = 60.

omuamua = pd.DataFrame({'name' : [name], 'family' : ['Oumuamua'], 'age_myr' : [age], 'U' : [U], 'V' : [V], 'W' : [W], 'x' : [x], 'y' : [y], 'z' : [z], 'n_stars' : [1]})

# Dark plot

In [301]:
# spurs
rw_data = StarClusterData(rw, data_name = 'Radcliffe Wave', min_size = 0, max_size = 9, color = 'red', opacity = .8, marker_style = 'circle', show_tracks = False)

# families
sample_data = StarClusterData(df_hunt_gagne, data_name = 'Parent Clusters Sample', min_size = 0, max_size = 7, color = 'gray', opacity = .8, marker_style = 'circle')
ap_data = StarClusterData(ap, data_name = '⍺Per', min_size = 0, max_size = 12, color = 'violet', opacity = .8, marker_style = 'circle')
ap_young_data = StarClusterData(ap_young, data_name = '⍺Per (< 15 Myr)', min_size = 0, max_size = 12, color = 'violet', opacity = .8, marker_style = 'circle')
m6_data = StarClusterData(m6, data_name = 'M6', min_size = 0, max_size = 12, color = 'cyan', opacity = .8, marker_style = 'circle')
cr135_data = StarClusterData(cr135, data_name = 'Cr135', min_size = 0, max_size = 12, color = 'orange', opacity = .8, marker_style = 'circle')
gamma_vel_data = StarClusterData(gamma_vel, data_name = 'Gamma Vel', min_size = 0, max_size = 9, color = 'red', opacity = .8, marker_style = 'circle')

young_data = StarClusterData(df_hunt_young, data_name = 'Clusters (< 10 Myr)', min_size = 0., max_size = 9, color = 'white', opacity = .8, marker_style = 'circle')
sun_data = StarClusterData(sun, data_name = 'Sun', min_size = 0, max_size = 8, color = 'yellow', opacity = 1, marker_style = 'circle')

#omuamua
omuamua_data = StarClusterData(omuamua, data_name = 'Oumuamua', min_size = 0, max_size = 20., color = 'pink', opacity = .8, marker_style = 'circle')

sc = StarClusterCollection([sample_data, sun_data, ap_data, m6_data, cr135_data, ap_young_data])
#sc = StarClusterCollection([cr135_data])
grouping_dict = None

In [ ]:
xyz_widths = (1100, 1100, 500)
#xyz_widths = (700, 700, 500)

for c in sc.get_all_clusters():
    if c.data_name not in ['Oumuamua', 'Sun'] and 'n_stars' in c.df.columns:
        c.size_by_n_stars = False
    if c.data_name in ['M6', '⍺Per', '⍺Per (< 15 Myr)', 'Cr135', 'Radcliffe Wave','Clean Clusters Sample', 'Young clusters', 'Oumuamua']:
        c.show_tracks = True
    #     c.size_by_n_stars = False
    # if c.data_name == 'Cr135':
    #     c.show_tracks = True


plot_3d = StarClusters3DPlotter(
    data_collection = sc, 
    xyz_widths = xyz_widths, 
    figure_theme = 'dark', 
    trace_grouping_dict=grouping_dict)

save_name = '/Users/cam/Desktop/astro_research/radcliffe/cam_website_clone/cam_website/interactive_figures/talk_dec3.html'
# save_name = '/Users/cam/Desktop/astro_research/radcliffe/cam_website_clone/cam_website/interactive_figures/talk_dec3_fade_in_and_out.html'
# save_name = '/Users/cam/Desktop/astro_research/radcliffe/cam_website_clone/cam_website/interactive_figures/talk_dec3_cr135_fade_in_and_out.html'
time_int = np.round(np.arange(0., -61, -1), 1)

fig3d = plot_3d.generate_3d_plot(
    time = time_int,
    show = False, 
    save_name =save_name, 
    static_traces = static_traces, 
    static_traces_times = static_traces_times, 
    static_traces_legendonly=True,
    focus_group = None,
    #focus_group = 'Cr135',
    fade_in_time = 5, # Myr,
    fade_in_and_out = False
)